Task 2:- STEP 1
Dataset Loading + Initial Exploration (EDA)

Load Dataset

We are using the IBM Telco Customer Churn Dataset, which is industry-standard and perfectly matches the internship requirement.

Dataset file name (important):

WA_Fn-UseC_-Telco-Customer-Churn.csv

Dataset Loading + Sanity Checks
Imports

In [8]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


STEP 2:- Read CSV Load Dataset

In [16]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


STEP 3:- Create X and y

In [9]:
X = df.drop(columns=["customerID", "Churn"])
y = df["Churn"]


STEP 4:- Identify Feature Types

In [17]:
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()
numerical_features = X.select_dtypes(exclude=["object"]).columns.tolist()

print("Categorical features:", categorical_features)
print("Numerical features:", numerical_features)


Categorical features: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges']
Numerical features: ['SeniorCitizen', 'tenure', 'MonthlyCharges']


STEP 5:- Train/Test Split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


STEP 6:- Numerical Pipeline

In [19]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


STEP 7:- Categorical Pipeline

In [20]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])


STEP 8:- ColumnTransformer

In [21]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


STEP 9:- Model Evaluation We will evaluate BOTH models using the same metrics so the comparison is fair.

We’ll: 

Predictions on X_test

Metrics: Accuracy, Precision, Recall, F1-score

Confusion Matrix

STEP 9.1:- Imports

In [25]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier



Define Logistic Regression pipeline

In [26]:
log_reg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])


Define Random Forest pipeline

In [27]:
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])


Train both models

In [28]:
log_reg_pipeline.fit(X_train, y_train)
rf_pipeline.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


STEP 9.2:- Predictions

In [29]:
# Logistic Regression predictions
y_pred_log = log_reg_pipeline.predict(X_test)

# Random Forest predictions
y_pred_rf = rf_pipeline.predict(X_test)


Evaluation Function

In [30]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

def evaluate_model(name, y_true, y_pred):
    print(f"\n===== {name} =====")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, pos_label="Yes"))
    print("Recall   :", recall_score(y_true, y_pred, pos_label="Yes"))
    print("F1-score :", f1_score(y_true, y_pred, pos_label="Yes"))
    print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))


STEP 9.3:- Evaluate Both Models

In [31]:
evaluate_model("Logistic Regression", y_test, y_pred_log)
evaluate_model("Random Forest", y_test, y_pred_rf)



===== Logistic Regression =====
Accuracy : 0.794180269694819
Precision: 0.63125
Recall   : 0.5401069518716578
F1-score : 0.5821325648414986

Confusion Matrix:
 [[917 118]
 [172 202]]

Classification Report:
               precision    recall  f1-score   support

          No       0.84      0.89      0.86      1035
         Yes       0.63      0.54      0.58       374

    accuracy                           0.79      1409
   macro avg       0.74      0.71      0.72      1409
weighted avg       0.79      0.79      0.79      1409


===== Random Forest =====
Accuracy : 0.7842441447835344
Precision: 0.625
Recall   : 0.4679144385026738
F1-score : 0.5351681957186545

Confusion Matrix:
 [[930 105]
 [199 175]]

Classification Report:
               precision    recall  f1-score   support

          No       0.82      0.90      0.86      1035
         Yes       0.62      0.47      0.54       374

    accuracy                           0.78      1409
   macro avg       0.72      0.68      0.70 

STEP 9:- INTERPRETATION Logistic Regression (Baseline)

Accuracy: ~79.4%

Recall (Churn = Yes): 0.54

F1-score: 0.58 Better at catching churners

Random Forest (Untuned)

Accuracy: ~78.4%

Recall (Churn = Yes): 0.47

F1-score: 0.54

Slightly worse at identifying churners in its default form.

Key Insight:- Even though Random Forest is more complex, Logistic Regression performs better on recall and F1, which are more important for churn prediction.

STEP 10:- Hyperparameter Tuning (GridSearchCV) 

STEP 10.1:- Import GridSearchCV

In [32]:
from sklearn.model_selection import GridSearchCV


STEP 10.2:- Parameter Grid (Focused & Safe)

In [33]:
param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 5]
}


STEP 10.3:- GridSearchCV Setup

In [34]:
grid_search = GridSearchCV(
    rf_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)


STEP 10.4:- Run Grid Search

In [35]:
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


c:\Users\HAROON UR RASHEED\.conda\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'classifier__max_depth': [None, 10, ...], 'classifier__min_samples_split': [2, 5], 'classifier__n_estimators': [100, 200]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


STEP 10.5:- Best Parameters & Model

In [36]:
print("Best parameters:", grid_search.best_params_)
print("Best CV F1-score:", grid_search.best_score_)

best_rf_model = grid_search.best_estimator_


Best parameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
Best CV F1-score: nan


STEP 11:- Evaluate the Tuned Random Forest

STEP 11.1:- Extract the Best Tuned Model

In [37]:
best_rf_model = grid_search.best_estimator_


STEP 11.2:- Generate Predictions on Test Set

In [38]:
y_pred_best_rf = best_rf_model.predict(X_test)


STEP 11.3:- Evaluate Tuned Random Forest

In [39]:
evaluate_model("Tuned Random Forest", y_test, y_pred_best_rf)



===== Tuned Random Forest =====
Accuracy : 0.7842441447835344
Precision: 0.625
Recall   : 0.4679144385026738
F1-score : 0.5351681957186545

Confusion Matrix:
 [[930 105]
 [199 175]]

Classification Report:
               precision    recall  f1-score   support

          No       0.82      0.90      0.86      1035
         Yes       0.62      0.47      0.54       374

    accuracy                           0.78      1409
   macro avg       0.72      0.68      0.70      1409
weighted avg       0.77      0.78      0.77      1409



Key Observation: Tuned Random Forest produced exactly the same performance as the untuned Random Forest.

That means:

GridSearch did not find better hyperparameters

The default model was already near-optimal 

Confusion Matrix Insight Main weakness:
The model misses many churn customers (high FN).

Best Model So Far

Logistic Regression
Because:

Higher Recall (important for churn)

Higher F1-score

More interpretable

STEP 12:- Final Model Selection

Selected Model: Logistic Regression

Reason:

“Since customer churn prediction prioritizes identifying as many churners as possible, Logistic Regression was selected due to its superior recall and F1-score, despite slightly lower interpretability trade-offs in tree-based models.”

STEP 13:- Improve Recall to level up the project, next options are:

1️⃣ Change classification threshold
2️⃣ Use class_weight='balanced'
3️⃣ ROC–AUC analysis
4️⃣ Business-cost-based evaluation

In churn prediction:

False Negatives (missed churners) are more costly than false positives

We want the model to catch more “Yes / Churn” cases

We will penalize mistakes on churn class using class_weight="balanced".

What class_weight="balanced" does

It automatically assigns higher weight to the minority class (Yes / Churn).

This forces the model to:

Pay more attention to churn customers

Increase Recall

Accept a small drop in Precision (this is expected & acceptable)

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Logistic Regression with class weights
log_reg_balanced = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        random_state=42
    ))
])

# Train
log_reg_balanced.fit(X_train, y_train)

# Predictions
y_pred_bal = log_reg_balanced.predict(X_test)


STEP 14:- Evaluate Weighted Logistic Regression

In [41]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print("===== Logistic Regression (Class Weighted) =====")
print("Accuracy :", accuracy_score(y_test, y_pred_bal))
print("Precision:", precision_score(y_test, y_pred_bal, pos_label="Yes"))
print("Recall   :", recall_score(y_test, y_pred_bal, pos_label="Yes"))
print("F1-score :", f1_score(y_test, y_pred_bal, pos_label="Yes"))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_bal))
print("\nClassification Report:\n", classification_report(y_test, y_pred_bal))


===== Logistic Regression (Class Weighted) =====
Accuracy : 0.7530163236337828
Precision: 0.5239852398523985
Recall   : 0.7593582887700535
F1-score : 0.6200873362445415

Confusion Matrix:
 [[777 258]
 [ 90 284]]

Classification Report:
               precision    recall  f1-score   support

          No       0.90      0.75      0.82      1035
         Yes       0.52      0.76      0.62       374

    accuracy                           0.75      1409
   macro avg       0.71      0.76      0.72      1409
weighted avg       0.80      0.75      0.76      1409



STEP 15:- What to Expect Accuracy goes down slightly

Precision decreases

What SHOULD improve:

Recall ↑

F1-score ↑

STEP 16:- Final Model Justification “A class-weighted Logistic Regression model was used to address class imbalance and improve recall for churn customers. Although precision slightly decreased, the model achieved higher recall and F1-score, making it more suitable for churn prediction where missing churners is costly.”